In [1]:
# import nltk

# # Download required NLTK data (only need to do this once)
# nltk.download('punkt')

# random notes 2-17 on algo improvement:

- "weight" articles parsed based on recency (more recent articles are weighted more heavily)
- news sources miss things social media has --> should hit some of those APIs
- can help daniel by creating targeted sora prompts 
- can help daniel by creating grok prompts for specific images


# Alterations to algo, 2/24
- shorter length per scene, but more scenes
- each scene should be more exciting/dramatic/crazy

# My thoughs on how to do that:
- make script with GROK not GPT-4o --> this will make asset generation downstream much easier

## AI VIDEO EDITOR // AI VIDEO EDITING WORKFLOW IS THE KEY MISSING PIECE TO A FULLY OPTIMIZED PIPELINE

In [2]:
import trendspy
from trendspy import Trends
import os
import datetime
from dotenv import load_dotenv
from news_utils import News
import feedparser
import time
import pickle
from generateAssets import *
load_dotenv()

c:\Users\garre\OneDrive\Desktop\Projects\aiMedia


True

In [3]:
# Make directory for the run
APPDIR = os.getenv("APPDIR")
BASEPATH = os.getenv("BASEPATH")
RUN_DIR = os.path.join(BASEPATH, datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
os.makedirs(BASEPATH, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("ELEVENLABS_VOICE_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.chdir(APPDIR)

In [4]:
os.environ["RUN_DIR"] = RUN_DIR

In [5]:
def getTrends():
    tr = Trends()
    trends = tr.trending_now(geo='US', hours=24)
    businessFiltered = trends.filter_by_topic('Business and Finance')
    techFiltered = trends.filter_by_topic('Technology')
    lawAndGovFiltered = trends.filter_by_topic('Law and Government')
    filteredTrends = businessFiltered + techFiltered + lawAndGovFiltered
    sortedTrends = sorted(filteredTrends, key=lambda x: x.volume, reverse=True)
    finalTrendList = []
    for trend in sortedTrends:
        finalTrendList.append({
            'keyword': trend.keyword,
            'volume': trend.volume,
            'volume_growth_pct': trend.volume_growth_pct,
            'topics': trend.topics,
            'keywords': trend.trend_keywords,
        })
    return finalTrendList

In [6]:
# Example usage in your getNews function:
def getNews(trendKeyword, num_articles=10, linkNews={}):
    search_term = trendKeyword.replace(' ', '+')
    url = f'https://news.google.com/rss/search?q={search_term}&hl=en-US&gl=US&ceid=US:en'
    feed = feedparser.parse(url)
    valid, invalid = [], []

    for entry in feed.entries[:min(num_articles, len(feed.entries))]:
        if entry.link not in linkNews:
            newNews = News(
                entry.title,
                entry.link,
                entry.published,
            )
            linkNews[entry.link] = newNews
            if newNews.mainText:
                valid.append(newNews)
            else:
                invalid.append(newNews)

    return valid, invalid

In [7]:
def collectArticles(trend, linkNews={}, maxArticles = 25):
    news = set()
    keywords = set(trend['keywords'] + trend['keywords'])
    numArticlesPerKeyword = 1 + (maxArticles // len(keywords)) # promotes diversity across keywords
    for keyword in keywords:
        time.sleep(2) # wait two seconds to avoid rate limiting
        valid, invalid = getNews(keyword, num_articles=numArticlesPerKeyword, linkNews=linkNews)
        news.update(valid)
        if len(news) > maxArticles:
            break
    trend['news'] = list(news)

# 🔄 Collect Trends Or Load From Pickle
---

In [8]:
getTrendInfo = input('Get Trends? (y/n): ')
if getTrendInfo.lower() == 'y':
    finalTrendList = getTrends()
    showTrends = input('Show Trends? (y/n): ')
    if showTrends.lower() == 'y':
        # outStr = """"""
        for trend in finalTrendList[:min(100, len(finalTrendList))]:
            print(trend)
            # outStr += f"**{trend['keyword']}**\n"
            # for k, v in trend.items():
            #     if k != 'keyword':
            #         outStr += f"{k} : {v}\n"
        # print(outStr)
    saveTrends = input('Save Trends? (y/n): ')
    if saveTrends.lower() == 'y':
        with open('./tmp/finalTrendList.pkl', 'wb') as f:
            pickle.dump(finalTrendList, f)
elif getTrendInfo.lower() == 'n':
    with open('./tmp/finalTrendList.pkl', 'rb') as f:
        finalTrendList = pickle.load(f)
else:
    print('Invalid input, run again')
    exit()

{'keyword': 'southwest airlines checked baggage fees', 'volume': 500000, 'volume_growth_pct': 800, 'topics': [3, 19], 'keywords': ['southwest airlines checked baggage fees', 'southwest airlines', 'southwest', 'southwest checked bags', 'southwest baggage fees', 'southwest checked bag', 'southwest airlines baggage', 'southwest airlines checked bags', 'southwest baggage policy', 'southwest bags', 'southwest charging for bags', 'southwest stock', 'southwest free bags', 'southwest credit card', 'southwest changes', 'southwest charge for bags', 'luv stock', 'southwest airlines baggage fees', 'southwest ending free bags', 'southwest free checked bag', 'southwest baggage', 'southwest bags fly free', 'southwest bag policy', 'southwest news']}
{'keyword': 'dow jones today', 'volume': 100000, 'volume_growth_pct': 100, 'topics': [3], 'keywords': ['dow jones today', 'nasdaq today', 'dow jones industrial average', 'dow today', 'stocks today', 'stock market today: live chart', 'dow jones', 'dow jones

# 🎯 Select Trend and Collect Articles
---

In [9]:
keyword = input('Enter trend keyword: ')
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]
linkNews = {}
collectArticles(trend, linkNews=linkNews, maxArticles = 40)
pickle.dump(finalTrendList, open('./tmp/finalTrendList.pkl', 'wb'))
print('Trends info scraped, trend selected and articles collected. All information saved.')

URL https://news.google.com/rss/articles/CBMiywFBVV95cUxQTTZ4WWpod1pjdkxhUDNxSUFISFpoMmVSdjRPanhON0c4SWVEUE5TbGoydGJ1UERqY1k5TmhSY2huS3dXbHZNNnBCMzg3WmlqOEh0WHk1dUFyT3hLMmNBR1YxQUFfdGE1SU9IQVNBUVpGclFKU042bTNSSjBwM0llNEx3S1VzMW9SbHFRTU9oaTliSGI3amZyV01NS3BIcFNUVHBnWFBXVDZSUFdrTjlDTUpCVjN0aTFTNkUtNlFnclRNSEdiM3pHVUpYWQ?oc=5
URL https://news.google.com/rss/articles/CBMi-wFBVV95cUxQZFg3MHcwY3BITmY3VGc3SnF3bHBhZGtZRGJ2RWZFdUI3ZzBXdFI2c2VGRzBzVWJLc1pHOHNQOUlsMmZvbHNweFN0MkNzRnVndUFpNThvYlZqUHlRRWNHU3g2XzBNQm83LTRLdkJSUTEzVmduRGt5Z3ExZUZtZ0dFS0pmaEpUd0NyNThST2NoU25NZUZ0QUk5R2RYczBVN3hPa2hQNzJnbnhLTGJjdVN5V0lMTy1WOHhyeTNJVHdCWDVJVUVib2xVclpGRVhmWl9TMExaXzNKR2x6N3NGVjRTb0ZzREZ3TW5Ccjdsa0tqN29yRnR5YTVwbXJyQQ?oc=5
URL https://news.google.com/rss/articles/CBMiggFBVV95cUxOaWJoQTlPSE00SjZILXExR29PN05FSzZYQ0QybC11UjBMenh4MjVHNTdyWHVyMmo1cFYwQTNCX2lSYjFRSVBSOTFZWHd0VnVtQTJQOHFLMmVNUllaQk5KSDNzZmtQMEl4RVhIbGdCdWhLTnBmODB0T0IyVVkxTi0xSGtR0gFUQVVfeXFMTm5zZmIwcGVvZ3JNUDRxN3RUcGVBTHBFeFVPbEhjYkFqaHZVN3FxV

# 🎨 Generate Assets For Selected Trend
---

In [10]:
config = {'news_report': True,
        'script': True,
        'narration': True,
        'voice': False,
        'sora_prompts': True}

In [14]:
print(ELEVENLABS_API_KEY)

sk_f96d619fc51235180796190a025b96954210e5b25af0d67d


In [12]:
generate_assets(trend, openai_api_key=OPENAI_API_KEY, elevenlabs_api_key=ELEVENLABS_API_KEY, voice_id=VOICE_ID, run_dir=RUN_DIR, config=config)

C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-11-21-22-47\department_of_education topic_dir
C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-11-21-22-47\department_of_education\sora_prompts sora_prompt_dir
28 validArticles


{'paths': {'script': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-11-21-22-47\\department_of_education\\script.txt',
  'audio': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-11-21-22-47\\department_of_education\\voiceover.mp3',
  'video': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-11-21-22-47\\department_of_education\\video.mp4',
  'sora_prompts': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-11-21-22-47\\department_of_education\\sora_prompts',
  'news_report': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-11-21-22-47\\department_of_education\\news_report.txt'}}

In [15]:
os.getenv("RUN_DIR")

'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-11-21-22-47'

In [16]:
os.chdir(RUN_DIR)
outName = trend['keyword'].replace(" ", "_") # name of folder and file to zip
!tar -acf {outName}.zip {outName}

In [17]:
!echo python -m wormhole send {outName}.zip | clip && start cmd